In [1]:
import pyvisa
import pandas as pd
import numpy as np
import time


In [2]:

rm = pyvisa.ResourceManager()
resources = rm.list_resources()
resource = resources[0]
spectrum_analyzer_resource = rm.open_resource(resource)

In [3]:
class SpectrumAnalyzer:
    def __init__(self, spectrum_analyzer_resource):
        self.spectrum_analyzer_resource = spectrum_analyzer_resource

    def get_id(self) -> str:
        return self.format_response(self.spectrum_analyzer_resource.query('*IDN?'))
    
    def set_center_frequency(self, frequency : float):
        self.ensure_range(0, 7.5e9, frequency)
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:CENTer {frequency}')
    
    def get_center_frequency(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SENSe:FREQuency:CENTer?')))
    
    def set_start_frequency(self, frequency : float):
        self.ensure_range(0, 7.5e9, frequency)
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:STARt {frequency}')
    
    def get_start_frequency(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SENSe:FREQuency:STARt?')))
    
    def set_stop_frequency(self, frequency : float):
        self.ensure_range(0, 7.5e9, frequency)
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:STOP {frequency}')

    def get_stop_frequency(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SENSe:FREQuency:STOP?')))
    
    def set_span(self, span : float):
        self.ensure_range(0, 7.5e9, span)
        self.spectrum_analyzer_resource.write(f'SENSe:FREQuency:SPAN {span}')

    def get_span(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SENSe:FREQuency:SPAN?')))

    def enable_tracking_generator(self):
        self.spectrum_analyzer_resource.write('OUTPut:STATe ON')

    def disable_tracking_generator(self):
        self.spectrum_analyzer_resource.write('OUTPut:STATe OFF')

    def is_tracking_generator_enable(self)-> bool:
        return self.format_response(self.spectrum_analyzer_resource.query('OUTPut:STATe?')) == '1'
    
    def set_tracking_generator_offset(self, level: int):
        self.ensure_range(-200, 200, level)
        self.spectrum_analyzer_resource.write(f'SOURce:CORRection:OFFSet {level}')

    def get_tracking_generator_offset(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SOURce:CORRection:OFFSet?')))
    
    def set_tracking_generator_level(self, level: int):
        self.ensure_range(-40, 0, level)
        self.spectrum_analyzer_resource.write(f'SOURce:POWer:LEVel:IMMediate:AMPLitude {level}')

    def get_tracking_generator_level(self) -> float:
        return float(self.format_response(self.spectrum_analyzer_resource.query('SOURce:POWer:LEVel:IMMediate:AMPLitude?')))

    def get_harmonics_frequency(self) -> np.array:
        return self.format_response(self.spectrum_analyzer_resource.query(':READ:HARMonics:FREQuency:ALL?'))
    
    def get_frequency_power(self, frequency) -> np.array:
        self.set_center_frequency(frequency)
        x = self.spectrum_analyzer_resource.query("TRACe:DATA? TRACE1")
        x = x.split(" ", 1)[-1]
        x = x.split(",")
        x = [float(a) for a in x]
        x = np.array(x)
        return np.median(x)
    
    def ensure_range(self, min, max, value):
        if value < min and value > max:
            raise ValueError(f'Value must be between {min} and {max}')
    
    def format_response(self, response):
        return response.strip()


In [4]:
spectrum_analyzer = SpectrumAnalyzer(spectrum_analyzer_resource)

In [ ]:
data = []

In [6]:
frequency = 100e3

while frequency < 3.2e9:
    gain = spectrum_analyzer.get_frequency_power(frequency)
    data.append({
        "frequency": frequency,
        "gain": gain,
        "plant": "lilium"
    })
    frequency = frequency + 10e6

In [7]:
df = pd.DataFrame(data)
df

In [21]:
df.to_csv("./data/noise_16_01_2025.csv")